In [55]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [43]:
# ファイルパス
train_path = "train.csv"
test_path = "test.csv"

# CSVファイルの読み込み
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# データの確認（先頭5行を表示）
print(train_df.head())
print(test_df.head())

          Id                                           sentence  polarity_flag
0  E05480-20  機器売上高は、前年同期比8.6％減の2,994百万円(前年同期 3,275百万円)、売上原価...              0
1   E01703-1  このような状況の中、当社グループは顧客満足度の向上を経営方針として掲げ、新たな価値の創出にチ...              0
2   E01946-7  平成28年度における連結業績は、売上高が5,369億42百万円（前期比13.9％減)、経常利...              0
3  E00457-21  日清食品チルド㈱の販売状況は、主力ブランド「行列のできる店のラーメン」を中心としたラーメン群...              1
4  E01743-15  設備工事等が増加したことにより、受注高は195億21百万円（前連結会計年度比15.8％増）と...              1
          Id                                           sentence
0  E00686-26                    このほか、セメント、製粉、砂糖・甘味、塩等の用途が減少しました
1  E01054-16  その結果、当連結会計年度の売上高は92億59百万円（前年同期比１億81百万円、2.0%増）と...
2   E04298-8  これに、特別損益を加減した税金等調整前当期純損失は128百万円（前年同期は276百万円の利益...
3  E21200-20  油圧機器につきましては、建設機械向けシリンダーは堅調に推移したものの、掘削機用ジャッキと免制...
4  E03991-10  マンション業界は、マイナス金利政策の導入や住宅取得税制の維持により、需要は堅調に推移しており...


In [44]:
train_df = train_df.drop(columns=["Id"])
test_df = test_df.drop(columns=["Id"])

In [45]:
train_df = train_df.dropna(subset=["sentence"])
test_df = test_df.dropna(subset=["sentence"])

In [46]:
import re

def clean_text(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    return text.strip()

train_df["sentence"] = train_df["sentence"].apply(clean_text)
test_df["sentence"] = test_df["sentence"].apply(clean_text)

In [47]:
!pip install mecab-python3 unidic-lite

In [48]:
import MeCab

mecab = MeCab.Tagger("-Owakati")

def tokenize(text):
    return mecab.parse(text).strip()

train_df["sentence"] = train_df["sentence"].apply(tokenize)
test_df["sentence"] = test_df["sentence"].apply(tokenize)

In [49]:
import MeCab

mecab = MeCab.Tagger("-Owakati")
text = "これは日本語の形態素解析のテストです。"
print(mecab.parse(text))

これ は 日本 語 の 形態 素 解析 の テスト です 。 



In [50]:
#
def tokenize(text):
    return mecab.parse(text).strip()

# データに適用
train_df["sentence"] = train_df["sentence"].apply(tokenize)
test_df["sentence"] = test_df["sentence"].apply(tokenize)

In [51]:
!pip install lightgbm

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF の計算
tfidf = TfidfVectorizer()
tfidf.fit(train_df["sentence"])
word_weights = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

# TF-IDF の重み付き Word2Vec ベクトル
def get_weighted_vector(sentence, model, weights):
    vectors = [model.wv[word] * weights.get(word, 1) for word in sentence if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

# 新しいベクトル作成
X_train_w2v_tfidf = np.vstack(train_tokens.apply(lambda x: get_weighted_vector(x, word2vec_model, word_weights)))
X_test_w2v_tfidf = np.vstack(test_tokens.apply(lambda x: get_weighted_vector(x, word2vec_model, word_weights)))

In [61]:
y_train = train_df["polarity_flag"]

# データ分割（学習データと検証データ）
X_train_split_tfidf, X_val_split_tfidf, y_train_split_tfidf, y_val_split_tfidf = train_test_split(
    X_train_w2v_tfidf, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# LightGBM モデルの作成（TF-IDF + Word2Vec ベクトルを使用）
lgbm_model_tfidf = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=10,
    num_leaves=50,
    min_child_samples=1,
    random_state=42
)

# モデルの学習
lgbm_model_tfidf.fit(X_train_split_tfidf, y_train_split_tfidf)

# 検証データでの予測
y_val_pred_tfidf = lgbm_model_tfidf.predict_proba(X_val_split_tfidf)[:, 1]

# AUCスコアの計算
auc_score_tfidf = roc_auc_score(y_val_split_tfidf, y_val_pred_tfidf)
print("TF-IDF + Word2Vec LightGBM AUC Score:", auc_score_tfidf)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 1324, number of negative: 733
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013453 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 2057, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.643656 -> initscore=0.591267
[LightGBM] [Info] Start training from score 0.591267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
TF-IDF + Word2Vec LightGBM AUC Score: 0.6712695389465388


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
